In [1]:
import requests
import pandas as pd
import datetime
from barcelona_keys import key as weather_key


def getting_daily_weather(year,month,day,variables_selected,codiEstacio='D5'):
    
    url=f'https://api.meteo.cat/xema/v1/estacions/mesurades/{codiEstacio}/{year}/{month}/{day}'
    response = requests.get(url, headers={"Content-Type": "application/json", "X-Api-Key": weather_key}).json()
    list_variables_codes=[variable['code'] for variable in variables_selected]
    list_variables_names=[variable['name'] for variable in variables_selected]
    df=pd.DataFrame()
    for variable in response[0]['variables']:
        if variable['codi'] in list_variables_codes:
            #print(variable)
            data=pd.DataFrame.from_records(variable['lectures'])[['data','valor']]
            data.rename(columns={'valor':list_variables_names[list_variables_codes.index(variable['codi'])]},inplace=True)

            if df.empty:
                df=data.copy()
            else:
                df=df.merge(data, on='data')
        
    return df


def getting_yearly_weather(year):
    start_date = datetime.date(year, 1, 1)
    end_date = datetime.date(year, 12, 31)
 
    # delta time
    delta = datetime.timedelta(days=1)
    yearly_weather=pd.DataFrame()
    while (start_date <= end_date):
        #print(start_date, end="\n")
        daily_weather=getting_daily_weather(year,'{:02d}'.format(start_date.month),'{:02d}'.format(start_date.day),variables_selected,codiEstacio='D5')
        yearly_weather=pd.concat([yearly_weather,daily_weather])

        start_date += delta
    yearly_weather.to_csv('./data/weatherbarcelona'+str(year)+'.csv',index=False)
    print("DONE",year)
    #return yearly_weather.reset_index(inplace=True)
    
    
##Figuring out estacions

url='https://api.meteo.cat/xema/v1/estacions/metadades?estat=ope&data=2023-06-01Z'
response = requests.get(url, headers={"Content-Type": "application/json", "X-Api-Key": weather_key})
estacions=response.json()
#estacions from Barcelona
estacions_barcelona=[{'code': estacio['codi'],'name':estacio['nom']} for estacio in estacions if estacio['municipi']['nom']=='Barcelona']

#figuring out variables
url='https://api.meteo.cat/xema/v1/variables/mesurades/metadades'
variables = requests.get(url, headers={"Content-Type": "application/json", "X-Api-Key": weather_key}).json()
selected_variables=[26,30,32,33,34,35,36,38]#selection done by going over all variables
variables_selected=[{'code':variable['codi'],'name':variable['nom'],'units':variable['unitat']} for variable in variables if variable['codi'] in selected_variables]

In [9]:
#single year
getting_yearly_weather(2011)

KeyError: 0

In [2]:
#doing each year for the fist time
first_year=2011
last_year=2022
for year in range(first_year,last_year+1):
    getting_yearly_weather(year)

KeyError: 0

In [6]:
url=f'https://api.meteo.cat/xema/v1/estacions/mesurades/D5/2011/01/01'
response = requests.get(url, headers={"Content-Type": "application/json", "X-Api-Key": weather_key}).json()

In [8]:
response

{'message': 'Limit Exceeded'}